In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [72]:
url_inicial = 'http://www.revistas.unam.mx/index.php/geofisica/issue/archive?issuesPage=6#issues'
url_root = 'http://www.revistas.unam.mx/index.php/geofisica/issue/archive?issuesPage=6#issues'

In [73]:
r=requests.get(url_inicial)

In [74]:
soup = BeautifulSoup(r.text, 'html.parser')

In [75]:
box = soup.find('div', id='issues')
volumen=box.findAll('div', style='float: left; width: 100%;')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['http://www.revistas.unam.mx/index.php/geofisica/issue/view/3119',
 'http://www.revistas.unam.mx/index.php/geofisica/issue/view/3115',
 'http://www.revistas.unam.mx/index.php/geofisica/issue/view/3110',
 'http://www.revistas.unam.mx/index.php/geofisica/issue/view/3108',
 'http://www.revistas.unam.mx/index.php/geofisica/issue/view/3107',
 'http://www.revistas.unam.mx/index.php/geofisica/issue/view/3106',
 'http://www.revistas.unam.mx/index.php/geofisica/issue/view/3104',
 'http://www.revistas.unam.mx/index.php/geofisica/issue/view/3101']

In [76]:
def get_url_items(sopa, url): 
    box = soup.find('div', id='issues')
    volumen=box.findAll('div', style='float: left; width: 100%;')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [ ]:
for i in vol: 
    #print(i)
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='obj_issue_toc') 
    volumen1=box1.findAll('div', class_='title')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol1
    for g in vol1: 
        print(g)

In [77]:
links_items=[]
i = 0
while i<6:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag = requests.get(url_inicial)
    s_p = BeautifulSoup(r_pag.text, 'html.parser')
    links = get_url_items(s_p, url_inicial)
    links_items.append(links)
    next_a = s_p.select('strong > a')
    if not next_a or not next_a[0].get('href'):
        break
        url_inicial = urljoin(url_inicial, next_a[0].get('href'))

Estoy en la pagina http://www.revistas.unam.mx/index.php/geofisica/issue/archive?issuesPage=6#issues


In [78]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

8

In [79]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [80]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Geofísica Internacional'
    a='I. Físico Matemáticas y Ciencias de la Tierra'
    tem='Sismología, Vulcanología, Ciencias Espaciales, Hidrología y Exploración, Paleomagnetismo y Tectónicas, Oceanografía Física'
    url='http://www.revistas.unam.mx/index.php/geofisica'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('a', class_='current').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('div', class_='journal-logo div_imagen_banner').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('div', id='issueCoverImage').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [81]:
list_scraper=list_scraper[0:8]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7


In [82]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Geofísica Internacional,I. Físico Matemáticas y Ciencias de la Tierra,"Sismología, Vulcanología, Ciencias Espaciales,...","Vol 13, No 4 (1973)",None,http://www.revistas.unam.mx/index.php/geofisica,http://www.revistas.unam.mx/index.php/geofisic...
1,Geofísica Internacional,I. Físico Matemáticas y Ciencias de la Tierra,"Sismología, Vulcanología, Ciencias Espaciales,...","Vol 12, No 4 (1972)",None,http://www.revistas.unam.mx/index.php/geofisica,http://www.revistas.unam.mx/index.php/geofisic...
2,Geofísica Internacional,I. Físico Matemáticas y Ciencias de la Tierra,"Sismología, Vulcanología, Ciencias Espaciales,...","Vol 11, No 3 (1971)",None,http://www.revistas.unam.mx/index.php/geofisica,http://www.revistas.unam.mx/index.php/geofisic...
3,Geofísica Internacional,I. Físico Matemáticas y Ciencias de la Tierra,"Sismología, Vulcanología, Ciencias Espaciales,...","Vol 10, No 2 (1970)",None,http://www.revistas.unam.mx/index.php/geofisica,http://www.revistas.unam.mx/index.php/geofisic...
4,Geofísica Internacional,I. Físico Matemáticas y Ciencias de la Tierra,"Sismología, Vulcanología, Ciencias Espaciales,...","Vol 8, No 2-4 (1968)",None,http://www.revistas.unam.mx/index.php/geofisica,http://www.revistas.unam.mx/index.php/geofisic...
5,Geofísica Internacional,I. Físico Matemáticas y Ciencias de la Tierra,"Sismología, Vulcanología, Ciencias Espaciales,...","Vol 7, No 3-4 (1967)",None,http://www.revistas.unam.mx/index.php/geofisica,http://www.revistas.unam.mx/index.php/geofisic...
6,Geofísica Internacional,I. Físico Matemáticas y Ciencias de la Tierra,"Sismología, Vulcanología, Ciencias Espaciales,...","Vol 5, No 4 (1965)",None,http://www.revistas.unam.mx/index.php/geofisica,http://www.revistas.unam.mx/index.php/geofisic...
7,Geofísica Internacional,I. Físico Matemáticas y Ciencias de la Tierra,"Sismología, Vulcanología, Ciencias Espaciales,...","Vol 1, No 2 (1961)",None,http://www.revistas.unam.mx/index.php/geofisica,http://www.revistas.unam.mx/index.php/geofisic...


In [83]:
df_catalogo.to_csv('Revista03.6.csv', index=False)